In [6]:
#VOC2012のデータセットを作成する
import torch.utils.data as data
import torch
import numpy as np
import cv2

class VOCDataset(data.Dataset):
    """
    VOC2012のDatasetを作成するクラス。PytorchのDatasetクラスを継承.
    
    Attributes
    ----------
    img_list : リスト
        画像のパスを格納したリスト
    anno_list : リスト
        アノテーションへのパスを格納したリスト
    phase : 'train' or 'test'<ー'val'の間違い？
        学習か訓練かを設定する
    transform : object
        前処理クラスのインスタンス
    transform_anno : object
        xmlのアノテーションをリストに変換するインスタンス
    """
    def __init__(self, img_list, anno_list, phase, transform, transform_anno):
        self.img_list = img_list
        self.anno_list = anno_list
        self.phase = phase # trainもしくはvalを指定。
        self.transform = transform #画像の変形
        self.transform_anno = transform_anno #アノテーションデータをxmlからリストへ
        
    def __len__(self):
        '''画像の枚数を返す'''
        return len(self.img_list)
    
    def __getitem__(self, index):
        '''
        前処理した画像のテンソル形式のデータとアノテーションを取得
        '''
        im, gt, h, w = self.pull_item(index)
        return im, gt
    
    def pull_item(self, index):
        '''
        前処理をした画像のテンソル形式のデータ、アノテーション、画像の高さ、幅を取得する
        '''
        #1.画像の読み込み
        image_file_path = self.img_list[index]
        img = cv2.imread(image_file_path) #[高さ][幅][色BGR]
        height, width, channels = img.shape #画像のサイズを取得
        
        #2.xml形式のアノテーション情報をリストに
        anno_file_path = self.anno_list[index]
        anno_list = self.transform_anno(anno_file_path, width, height)
        
        #3.前処理の実施
        img, boxes, labels = self.transform(
            img, self.phase, anno_list[:, :4], anno_list[:, 4])
        
        #色チャネルの順番がBGRになっているので、RGBに順番変更
        #さらに、(高さ、幅、色チャネル)の順を(色チャネル、高さ、幅)に変換
        img = torch.from_numpy(img[:, :, (2, 1, 0)]).permute(2,0,1)
        
        #BBoxとラベルをセットにしたnp.arrayを作成、変数名gtはground truth(答え)の略称
        gt = np.hstack((boxes, np.expand_dims(labels, axis=1)))
        
        return img, gt, height, width
        

In [10]:
#動作確認
from make_datapath_list import make_datapath_list
from Anno_xml2list import Anno_xml2list
from DataTransform import DataTransform

color_mean = (104, 117, 123) #(BGR)の色の平均値
input_size = 300  #画像のサイズを300*300にする

rootpath = "./data/VOCdevkit/VOC2012/"
train_img_list, train_anno_list, val_img_list, val_anno_list = make_datapath_list(rootpath)

voc_classes = ['aeroplane', 'bicycle', 'bird', 'boat',
               'bottle', 'bus', 'car', 'cat', 'chair',
               'cow', 'diningtable', 'dog', 'horse',
               'motorbike', 'person', 'pottedplant',
               'sheep', 'sofa', 'train', 'tvmonitor']

train_dataset = VOCDataset(train_img_list, train_anno_list, phase="train", transform = DataTransform(input_size, color_mean), transform_anno = Anno_xml2list(voc_classes))
val_dataset = VOCDataset(val_img_list, val_anno_list, phase="val", transform = DataTransform(input_size, color_mean), transform_anno = Anno_xml2list(voc_classes))

#データの取り出し例
val_dataset.__getitem__(1)

train
person


(tensor([[[   0.9417,    6.1650,   11.1283,  ...,  -22.9083,  -13.2200,
             -9.4033],
          [   6.4367,    9.6600,   13.8283,  ...,  -21.4433,  -18.6500,
            -18.2033],
          [  10.8833,   13.5500,   16.7000,  ...,  -20.9917,  -24.5250,
            -25.1917],
          ...,
          [ -23.9500,  -14.9000,   -1.7583,  ..., -108.6083, -111.0000,
           -117.8083],
          [ -28.2817,  -20.1750,   -5.5633,  ..., -104.9933, -111.8350,
           -119.0000],
          [ -20.4767,  -21.0000,  -12.6333,  ..., -107.1683, -115.7800,
           -117.1100]],
 
         [[  25.9417,   30.1650,   35.1283,  ...,  -18.0767,  -14.7250,
            -11.8533],
          [  31.4367,   33.6600,   37.8283,  ...,  -13.5017,  -10.8250,
            -10.3783],
          [  35.7917,   37.5500,   40.7000,  ...,  -11.8417,  -13.0750,
            -14.0167],
          ...,
          [  -1.9500,    7.1000,   20.2417,  ..., -101.9083, -102.0000,
           -109.7167],
          [  -6.2